# Marine Hydrokinetic RM2

In this example we use the OWENS scripting method to model the RM2 turbine,
based on “Experimental Study of a Reference Model Vertical-Axis Cross-Flow Turbine”, 2016, Bachant et al.
While the windio modeling option based on input yaml files is also included in the OWENS.jl/examples directory,
The scripting method enable much more flexibility and the ability to add detailed comments on the setup and use.
Note that the documentation contains an API reference for nearly all of the functions where more details can be found
also accessed via the julia REPL by loading the module of interest, and: ? ModuleName.functionname()

![](../assets/RM2_wake.png)

Load in all of the modules that we'll be using, and prettify the plotting options

In [1]:
import OWENS
import OWENSAero
import DelimitedFiles
using Statistics:mean
using Test
import FLOWMath

# import PyPlot
# PyPlot.pygui(true)
# PyPlot.rc("figure", figsize=(4.5, 3))
# PyPlot.rc("font", size=10.0)
# PyPlot.rc("lines", linewidth=1.5)
# PyPlot.rc("lines", markersize=3.0)
# PyPlot.rc("legend", frameon=false)
# PyPlot.rc("axes.spines", right=false, top=false)
# PyPlot.rc("figure.subplot", left=.18, bottom=.17, top=0.9, right=.9)
# PyPlot.rc("figure",max_open_warning=500)
# plot_cycle=["#348ABD", "#A60628", "#009E73", "#7A68A6", "#D55E00", "#CC79A7"]

# path = runpath = splitdir(@__FILE__)[1]
runpath = path = "/home/runner/work/OWENS.jl/OWENS.jl/examples/literate" # to run locally, change to splitdir(@__FILE__)[1]

nothing

Here we set the parametric inputs to the preprocessing and run methods.  Defaults are used otherwise
Note that for the OpenFAST coupling, such as adi_lib, setting these to nothing will force the program
to use the precompiled binaries that are part of the OWENS installation.
Again, please refer the to API reference for more detailed options of inputs, but of note is that we are
running with AeroModel = "AD" which will automatically generate all of the AeroDyn files necessary to run the
structural definition generated here.  Also note that this scripting method relys on the NuMAD formatted CSV files
for the composite layup that gets run through PreComp.jl to calculate the sectional properties. Finally, this
has been shortened to enable automated deployment, update as desired.

In [2]:
turbineType = "H-VAWT" # turbine type, for the automatic meshing
Vinf = 1.2 # inflow velocity
TSRrange = [3.0]#LinRange(1.0,5.0,2) range of tip speed ratios
Nslices = 20 # vertical discretizations if DMS or AC aero model
ntheta = 30 # azimuthal discretizations if DMS or AC aero model
structuralModel = "GX"
ntelem = 100 # tower elements
nbelem = 30 # blade elements
nselem = 10 # strut elements
ifw = false # use inflow wind, if DMS or AC aero model
numTS = 20#321 # number of simulation time steps
delta_t = 0.01 # simulation time step spacing
adi_lib = nothing#"$path/../../../../openfast/build/modules/aerodyn/libaerodyn_inflow_c_binding"
adi_rootname = "$path/RM2" # path and name that all the aerodyn files are saved with
VTKsaveName = "$path/vtk/RM2_medium" # path and name that all OWENS VTK files are saved with
tsave_idx = 1:1:numTS-1 #you don't have to save every timestep in VTK
ifw_libfile = nothing#"$path/../../../openfast/build/modules/inflowwind/libifw_c_binding"
fluid_density = 1000.0
fluid_dyn_viscosity = 1.792E-3
AddedMass_Coeff_Ca = 1.0 #For structural side added mass
Aero_Buoyancy_Active = true # For buoyancy forcing, handled by the OWENSAero module
AeroModel = "AD"
verbosity = 1 # verbosity level where higher is more
if AeroModel=="AD"
    AD15On = true
else
    AD15On = false
end

eta = 0.5 # blade-strut mount point ratio, fraction from leading edge
number_of_blades = 3 # number of blades
towerHeight = 0.2165 # height of the tower past the blades
Blade_Height = 0.807 # height of the blades
Blade_Radius = 0.5375 # radius of the turbine

NuMad_geom_xlscsv_file_twr = "$path/data_RM2/TowerGeom.csv" #
NuMad_mat_xlscsv_file_twr = "$path/data_RM2/TowerMaterials.csv"
NuMad_geom_xlscsv_file_bld = "$path/data_RM2/GeomBlades$AeroModel.csv"
NuMad_mat_xlscsv_file_bld = "$path/data_RM2/materials_NuMAD.csv"
NuMad_geom_xlscsv_file_strut = "$path/data_RM2/GeomStruts$AeroModel.csv"
NuMad_mat_xlscsv_file_strut = "$path/data_RM2/materials_NuMAD.csv"

nothing

For this simulation, using aerodyn ("AD"), we will use a turbulent inflow file, indicated as WindType 3
We can let the built in OWENS library for turbsim generate the file as so. Modify the inp file to your liking and
comment out the run command to run your own with more time

In [3]:
WindType = 3
windINPfilename = "$path/data_RM2/3mx3m1pt2msNTM.bts"
run(`$(OWENS.OWENSOpenFASTWrappers.turbsim()) $(windINPfilename[1:end-3])inp`)

nothing


 **************************************************************************************************
 TurbSim

 Copyright (C) 2024 National Renewable Energy Laboratory
 Copyright (C) 2024 Envision Energy USA LTD

 This program is licensed under Apache License Version 2.0 and comes with ABSOLUTELY NO WARRANTY.
 See the "LICENSE" file distributed with this software for details.
 **************************************************************************************************

  Running TurbSim (from OpenFAST-v2.5.0-3490-g24c05a74)


  Reading the input file
  "/home/runner/work/OWENS.jl/OWENS.jl/examples/literate/data_RM2/3mx3m1pt2msNTM.inp".
 TS_ValidateInput:A coherent turbulence time step file cannot be generated with the IECKAI model.
  Calculating the spectral and transfer function matrices:
     u-component matrices (IEC coherence method)
     v-component matrices (identity coherence)
     w-component matrices (identity coherence)
  Generating time series for all points:
     u-co

Here we would set up to run multiple different inflow conditions, and we can
allow the preprocessing to re-run each time since it only takes a few seconds.
The for loop is commented out to enable this example to be parsed by the literate
package that auto generates the docs.

In [4]:
CP = zeros(length(TSRrange))
mymesh = []
myel = []
system = []
assembly = []
sections = []
myjoint = []
pBC = []
# for (iTSR,TSR) in enumerate(collect(TSRrange))
TSR = TSRrange[1]
iTSR = 1
    global Vinf
    global mymesh
    global myel
    global system
    global assembly
    global sections
    global myjoint
    global pBC

    omega = Vinf/Blade_Radius*TSR
    RPM = omega * 60 / (2*pi)

    println(RPM)

    nothing

63.958079456463985


Here we run the OWENS preprocessing, which creates the beam mesh and joints, calculates the sectional properties
maps the sectional properties to the elements.  It also sets up the aero module being used.

In [5]:
    shapeZ = collect(LinRange(0,Blade_Height,Nslices+1))
    helix_angle = 0.0#-pi/4
    shapeX = cos.(shapeZ/maximum(shapeZ)*helix_angle).*Blade_Radius
    shapeY = sin.(shapeZ/maximum(shapeZ)*helix_angle).*Blade_Radius

    mymesh,myel,myort,myjoint,sectionPropsArray,mass_twr, mass_bld,
    stiff_twr, stiff_bld,bld_precompinput,
    bld_precompoutput,plyprops_bld,numadIn_bld,lam_U_bld,lam_L_bld,
    twr_precompinput,twr_precompoutput,plyprops_twr,numadIn_twr,lam_U_twr,lam_L_twr,aeroForces,deformAero,
    mass_breakout_blds,mass_breakout_twr,system,assembly,sections,AD15bldNdIdxRng, AD15bldElIdxRng = OWENS.setupOWENS(OWENSAero,path;
        rho=fluid_density,
        mu=fluid_dyn_viscosity,
        Nslices,
        ntheta,
        RPM,
        Vinf,
        eta,
        B=number_of_blades,
        H = Blade_Height,
        R = Blade_Radius,
        shapeZ,
        shapeX,
        shapeY,
        ifw,
        WindType,
        delta_t,
        numTS,
        adi_lib,
        adi_rootname,
        windINPfilename,
        ifw_libfile,
        NuMad_geom_xlscsv_file_twr,
        NuMad_mat_xlscsv_file_twr,
        NuMad_geom_xlscsv_file_bld,
        NuMad_mat_xlscsv_file_bld,
        NuMad_geom_xlscsv_file_strut,
        NuMad_mat_xlscsv_file_strut,
        Htwr_base=towerHeight,
        Htwr_blds = Blade_Height+towerHeight,
        ntelem,
        nbelem,
        nselem,
        joint_type = 0,
        strut_twr_mountpoint = [0.5],
        strut_bld_mountpoint = [0.5],
        AeroModel, #AD, DMS, AC
        DynamicStallModel="BV",
        Aero_AddedMass_Active = false,
        Aero_RotAccel_Active = false,
        AddedMass_Coeff_Ca,
        Aero_Buoyancy_Active,
        verbosity,
        meshtype = turbineType)

    nothing

    # This plots the mesh and node numbering of the resulting mesh and overlays the joint connections

    # PyPlot.figure()
    # for icon = 1:length(mymesh.conn[:,1])
    #     idx1 = mymesh.conn[icon,1]
    #     idx2 = mymesh.conn[icon,2]
    #     PyPlot.plot3D([mymesh.x[idx1],mymesh.x[idx2]],[mymesh.y[idx1],mymesh.y[idx2]],[mymesh.z[idx1],mymesh.z[idx2]],"k.-")
    #     PyPlot.plot3D([1,1],[1,1],[1,1],"k.-")
    #     PyPlot.text3D(mymesh.x[idx1].+rand()/30,mymesh.y[idx1].+rand()/30,mymesh.z[idx1].+rand()/30,"$idx1",ha="center",va="center")
    # end
    # for ijoint = 1:length(myjoint[:,1])
    #     idx2 = Int(myjoint[ijoint,2])
    #     idx1 = Int(myjoint[ijoint,3])
    #     PyPlot.plot3D([mymesh.x[idx1],mymesh.x[idx2]],[mymesh.y[idx1],mymesh.y[idx2]],[mymesh.z[idx1],mymesh.z[idx2]],"r.-")
    #     PyPlot.text3D(mymesh.x[idx1].+rand()/30,mymesh.y[idx1].+rand()/30,mymesh.z[idx1].+rand()/30,"$idx1",color="r",ha="center",va="center")
    #     PyPlot.text3D(mymesh.x[idx2].+rand()/30,mymesh.y[idx2].+rand()/30,mymesh.z[idx2].+rand()/30,"$idx2",color="r",ha="center",va="center")
    # end
    # PyPlot.xlabel("x")
    # PyPlot.ylabel("y")
    # PyPlot.zlabel("z")
    # PyPlot.axis("equal")

    nothing

┌ Warning: Mesh warning: two points are directly on top of one another, consider adjusting number of elements to space out the mesh
└ @ OWENS ~/work/OWENS.jl/OWENS.jl/src/meshing_utilities.jl:361
┌ Warning: Mesh warning: two points are directly on top of one another, consider adjusting number of elements to space out the mesh
└ @ OWENS ~/work/OWENS.jl/OWENS.jl/src/meshing_utilities.jl:392
Hub crossing at idx 3 and radially at -0.007499999999999993 with AD15 hub radius of 0.1
Moving strut point from [-0.1,6.123233995736766e-18,0.6199998498604652] to [-0.1,6.123233995736766e-18,0.6199998498604652]
Hub crossing at idx 3 and radially at -0.007499999999999993 with AD15 hub radius of 0.1
Moving strut point from [0.04999999999999998,0.08660254037844388,0.6199998498604652] to [0.04999999999999998,0.08660254037844388,0.6199998498604652]
Hub crossing at idx 3 and radially at -0.007499999999999993 with AD15 hub radius of 0.1
Moving strut point from [0.05000000000000004,-0.08660254037844387,0.6199

Here we apply the boundary conditions.  For this case, the tower base node which is
1 is constrained in all 6 degrees of freedom to have a displacement of 0, and the tower top is also constrained.
You can change this displacement to allow for things like pretension, and you can apply boundary conditions to any node.

In [6]:
    pBC = [1 1 0
    1 2 0
    1 3 0
    1 4 0
    1 5 0
    1 6 0
    AD15bldElIdxRng[1,2]-1 1 0
    AD15bldElIdxRng[1,2]-1 2 0
    AD15bldElIdxRng[1,2]-1 3 0
    AD15bldElIdxRng[1,2]-1 4 0
    AD15bldElIdxRng[1,2]-1 5 0]

    nothing

There are inputs for the overall coupled simulation, please see the api reference for specifics on all the options: ? OWENS.Inputs()

In [7]:
    tocp = [0.0;10.0; 1e6]
    Omegaocp = [RPM; RPM; RPM]./60 #control inputs

    tocp_Vinf = [0.0;10.0; 1e6]
    Vinfocp = [Vinf;Vinf;Vinf]

    inputs = OWENS.Inputs(;verbosity,analysisType = structuralModel,
    tocp,
    dataOutputFilename = "./InitialDataOutputs_scripting.out",
    Omegaocp,
    tocp_Vinf,
    Vinfocp,
    numTS,
    delta_t,
    AD15On,
    aeroLoadsOn = 2,
    MAXITER = 20)

    nothing

Then there are inputs for the finite element models, also, please see the api reference for specifics on the options

In [8]:
    FEAinputs = OWENS.FEAModel(;analysisType = structuralModel,
    dataOutputFilename = "./InitialDataOutputs_scripting.out",
    joint = myjoint,
    platformTurbineConnectionNodeNumber = 1,
    pBC,
    nlOn = false,
    gravityOn = [0,0,9.81], #positive since the turbine is upside down
    numNodes = mymesh.numNodes,
    RayleighAlpha = 0.05,
    RayleighBeta = 0.05,
    AddedMass_Coeff_Ca,
    iterationType = "DI")

    nothing

Here is where we actually call the unsteady simulation and where owens pulls the aero and structural solutions together
and propogates things in time. Note that for the sake of quickly deployable docs, a shortened simulation is conducted and the CP will not be correct

In [9]:
    println("Running Unsteady")
    t, aziHist,OmegaHist,OmegaDotHist,gbHist,gbDotHist,gbDotDotHist,FReactionHist,
    FTwrBsHist,genTorque,genPower,torqueDriveShaft,uHist,uHist_prp,epsilon_x_hist,epsilon_y_hist,
    epsilon_z_hist,kappa_x_hist,kappa_y_hist,kappa_z_hist,FPtfmHist,FHydroHist,FMooringHist,
    topFexternal_hist,rbDataHist = OWENS.Unsteady_Land(inputs;system,assembly,
    topModel=FEAinputs,topMesh=mymesh,topEl=myel,aero=aeroForces,deformAero,turbsimfile = windINPfilename)

    area = Blade_Height*2*Blade_Radius
    full_rev_N_timesteps = round(Int,RPM/60/delta_t)
    if full_rev_N_timesteps>numTS
        idx_start = 1
    else
        idx_start = numTS-full_rev_N_timesteps
    end
    CP[iTSR] = mean(FReactionHist[idx_start:end,6].*OmegaHist[idx_start:end]*2*pi)/(0.5*fluid_density*mean(Vinfocp)^3*area)
    TSR = mean(OmegaHist*2*pi*Blade_Radius/mean(Vinfocp))
    ReD = fluid_density*mean(Vinfocp)*Blade_Radius*2/fluid_dyn_viscosity

    nothing

Running Unsteady
Running in specified rotor speed mode

5.3%┣██▎                                        ┫ 1/19 [00:00<Inf:Inf, InfGs/it]
 [INFO] FVW: Update States: reevaluation at the same starting time.  This will not print on
 subsequent occurences.
10.5%┣█████                                          ┫ 2/19 [00:03<00:57, 3s/it]
15.8%┣███████▍                                       ┫ 3/19 [00:04<00:34, 2s/it]
21.1%┣██████████                                     ┫ 4/19 [00:05<00:27, 2s/it]
26.3%┣████████████▍                                  ┫ 5/19 [00:07<00:23, 2s/it]
31.6%┣██████████████▉                                ┫ 6/19 [00:08<00:22, 2s/it]
36.8%┣█████████████████▎                             ┫ 7/19 [00:10<00:19, 2s/it]
┌ Warning: Maximum Iterations Met Breaking Iteration Loop
└ @ OWENS ~/work/OWENS.jl/OWENS.jl/src/Unsteady_Land.jl:505
42.1%┣███████████████████▉                           ┫ 8/19 [00:14<00:22, 2s/it]
┌ Warning: Maximum Iterations Met Breaking Iteration Loop
└ @ 

We can output vtk files viewable in paraview.  Here it is done for each time step and shows the
deformations.  Additionaly, there is a method to input custom values and have them show up on the vtk surface mesh
for example, strain, or reaction force, etc.  This is described in more detail in the api reference for the function
This helper function looks through all the loads and picks out the worst case safety factor in each of the stacks of composite lamina
it also calculates analytical simply supported buckling safety factors

In [10]:
    ##########################################
    #### Ultimate Failure #####
    ##########################################

    massOwens,stress_U,SF_ult_U,SF_buck_U,stress_L,SF_ult_L,SF_buck_L,stress_TU,SF_ult_TU,
    SF_buck_TU,stress_TL,SF_ult_TL,SF_buck_TL,topstrainout_blade_U,topstrainout_blade_L,
    topstrainout_tower_U,topstrainout_tower_L,topDamage_blade_U,
    topDamage_blade_L,topDamage_tower_U,topDamage_tower_L = OWENS.extractSF(bld_precompinput,
    bld_precompoutput,plyprops_bld,numadIn_bld,lam_U_bld,lam_L_bld,
    twr_precompinput,twr_precompoutput,plyprops_twr,numadIn_twr,lam_U_twr,lam_L_twr,
    mymesh,myel,myort,number_of_blades,epsilon_x_hist,kappa_y_hist,kappa_z_hist,epsilon_z_hist,
    kappa_x_hist,epsilon_y_hist;verbosity, #Verbosity 0:no printing, 1: summary, 2: summary and spanwise worst safety factor
    Twr_LE_U_idx=1,Twr_LE_L_idx=1,
    AD15bldNdIdxRng,AD15bldElIdxRng,strut_precompoutput=nothing)

    OWENS.outputData(;mymesh,inputs,t,aziHist,OmegaHist,OmegaDotHist,gbHist,gbDotHist,gbDotDotHist,FReactionHist,genTorque,genPower,torqueDriveShaft,uHist,uHist_prp,epsilon_x_hist,epsilon_y_hist,epsilon_z_hist,kappa_x_hist,kappa_y_hist,kappa_z_hist,FTwrBsHist,massOwens,stress_U,SF_ult_U,SF_buck_U,stress_L,SF_ult_L,SF_buck_L,stress_TU,SF_ult_TU,SF_buck_TU,stress_TL,SF_ult_TL,SF_buck_TL,topstrainout_blade_U,topstrainout_blade_L,topstrainout_tower_U,topstrainout_tower_L,topDamage_blade_U,topDamage_blade_L,topDamage_tower_U,topDamage_tower_L)

    OWENS.OWENSVTK(VTKsaveName,t,uHist,system,assembly,sections,aziHist,mymesh,myel,
        epsilon_x_hist,epsilon_y_hist,epsilon_z_hist,kappa_x_hist,kappa_y_hist,kappa_z_hist,
        FReactionHist,topFexternal_hist;tsave_idx)


# end

# PyPlot.figure("CP2")
# PyPlot.plot(TSRrange,CP,".-",color=plot_cycle[2],label="OWENS Aero, 1.3 RE_d (No Added Mass)") #,color=color_cycle[2]
# RM2_0_538D_RE_D_1_3E6 = DelimitedFiles.readdlm("$(path)/data_RM2/RM2_0.538D_RE_D_1.3E6.csv", ',',Float64)
# PyPlot.plot(RM2_0_538D_RE_D_1_3E6[:,1],RM2_0_538D_RE_D_1_3E6[:,2],"k-",label="Exp. 1.3e6 RE_d")
# PyPlot.legend()
# PyPlot.xlabel("TSR")
# PyPlot.ylabel("Cp")

nothing

Composite Ultimate and Buckling Safety Factors


UPPER BLADE SURFACE

Minimum Safety Factor on Surface: 24.32695826140215
At time 0.019s at composite station 3 of 5 at lam 4 of 6
Maximum Damage per hr: 3.504765559638529e-13
At composite station 3 of 5 at lam 4 of 6


LOWER BLADE SURFACE

Minimum Safety Factor on Surface: 40.00753027644186
At time 0.019s at composite station 3 of 5 at lam 4 of 6
Maximum Damage per hr: 1.5579484640732923e-13
At composite station 3 of 5 at lam 4 of 6


UPPER TOWER

Minimum Safety Factor on tower Surface: 22.829376804374792
At time 0.002s at composite station 9 of 15 at lam 1 of 1
Maximum Damage per hr: 5.276868555907649e-11
At composite station 2 of 15 at lam 1 of 1


Lower TOWER

Minimum Safety Factor on tower Surface: 67.04428675312457
At time 0.019s at composite station 7 of 15 at lam 1 of 1
Maximum Damage per hr: 1.3745171137006024e-13
At composite station 8 of 15 at lam 1 of 1
WRITING Output File to ./InitialDataOutputs_scripting.out
Saving VTK time 

Here we use the automated campbell diagram function to run the modal analysis of the turbine and save the modeshapes to VTK

In [11]:
rotSpdArrayRPM = [0.0, 42.64]

FEAinputs = OWENS.FEAModel(;analysisType = "GX",
dataOutputFilename = "none",
joint = myjoint,
platformTurbineConnectionNodeNumber = 1,
pBC,
nlOn = false,
gravityOn = [0,0,9.81], #positive since the turbine is upside down
numNodes = mymesh.numNodes,
RayleighAlpha = 0.05,
RayleighBeta = 0.05,
AddedMass_Coeff_Ca,
iterationType = "DI")

freq2 = OWENS.AutoCampbellDiagram(FEAinputs,mymesh,myel,system,assembly,sections;
    rotSpdArrayRPM,
    VTKsavename=VTKsaveName,
    saveModes = [1,3,5], #must be int
    saveRPM = [2], #must be int
    mode_scaling = 500.0,
    )
freqGX = [freq2[:,i] for i=1:2:FEAinputs.numModes-6-2]

nothing

# Now the Campbell diagram can be generated
# NperRevLines = 8
# PyPlot.figure()
# for i=1:NperRevLines
#     linex=[rotSpdArrayRPM[1], rotSpdArrayRPM[end]+5]
#     liney=[rotSpdArrayRPM[1], rotSpdArrayRPM[end]+5].*i./60.0
#     PyPlot.plot(linex,liney,"--k",linewidth=0.5)
#     PyPlot.annotate("$i P",xy=(0.95*linex[2],liney[2]+.05+(i-1)*.01))
# end
# PyPlot.grid()
# PyPlot.xlabel("Rotor Speed (RPM)")
# PyPlot.ylabel("Frequency (Hz)")
# PyPlot.plot(0,0,"k-",label="Experimental")
# PyPlot.plot(0,0,color=plot_cycle[1],"-",label="OWENS")
# PyPlot.legend()

1 of 2
2 of 2
Saving Modeshapes to VTK


In [12]:
# for i=1:1:FEAinputs.numModes
#        PyPlot.plot(rotSpdArrayRPM,freq2[:,i],color=plot_cycle[2],"-") #plot mode i at various rotor speeds
# end
# PyPlot.plot(0,0,color=plot_cycle[2],"-",label="GXBeam")
# PyPlot.legend(fontsize=8.5,loc = (0.09,0.8),ncol=2,handleheight=1.8, labelspacing=0.03)
# PyPlot.ylim([0,40.0])

nothing

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*